# Daily Trends Prediction - LightGBM Model

This notebook trains a LightGBM model to predict daily interest values for Google Trends data.

**Algorithm**: LightGBM (Light Gradient Boosting Machine)
- Fast training speed
- Low memory usage
- High accuracy
- Handles categorical features natively

**Target**: Predict interest_value (0-100) for next day

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully")

## 2. Load Processed Data

In [ ]:
# Load processed daily trends data
# Replace with your actual processed file path
data_path = '../data/processed/daily_trends_processed_latest.csv'

df = pd.read_csv(data_path)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['keyword', 'category', 'date'])

print(f"Data shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Keywords: {df['keyword'].nunique()}")
print(f"Categories: {df['category'].nunique()}")

df.head()

## 3. Feature Engineering

In [ ]:
def create_features(df):
    """
    Create time-based and lag features for LightGBM
    """
    df = df.copy()
    
    # Time-based features
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['weekofyear'] = df['date'].dt.isocalendar().week
    
    # Lag features (previous days)
    for lag in [1, 2, 3, 7, 14, 30]:
        df[f'lag_{lag}'] = df.groupby(['keyword', 'category'])['interest_value'].shift(lag)
    
    # Rolling statistics
    for window in [7, 14, 30]:
        df[f'rolling_mean_{window}'] = df.groupby(['keyword', 'category'])['interest_value'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f'rolling_std_{window}'] = df.groupby(['keyword', 'category'])['interest_value'].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
    
    # Trend features
    df['diff_1'] = df.groupby(['keyword', 'category'])['interest_value'].diff(1)
    df['diff_7'] = df.groupby(['keyword', 'category'])['interest_value'].diff(7)
    
    return df

# Create features
df = create_features(df)

# Drop rows with NaN (from lag features)
df = df.dropna()

print(f"Features created. New shape: {df.shape}")
print(f"Feature columns: {df.shape[1]}")

## 4. Prepare Train/Test Split

In [ ]:
# Define features and target
feature_cols = [
    'day', 'month', 'year', 'dayofweek', 'quarter', 'weekofyear',
    'lag_1', 'lag_2', 'lag_3', 'lag_7', 'lag_14', 'lag_30',
    'rolling_mean_7', 'rolling_mean_14', 'rolling_mean_30',
    'rolling_std_7', 'rolling_std_14', 'rolling_std_30',
    'diff_1', 'diff_7'
]

categorical_features = ['keyword', 'category', 'dayofweek', 'month', 'quarter']

# Encode categorical features
for col in categorical_features:
    if col in df.columns:
        df[col] = df[col].astype('category')

# Time-based split (80% train, 20% test)
split_date = df['date'].quantile(0.8)

train_df = df[df['date'] < split_date]
test_df = df[df['date'] >= split_date]

X_train = train_df[feature_cols + categorical_features]
y_train = train_df['interest_value']

X_test = test_df[feature_cols + categorical_features]
y_test = test_df['interest_value']

print(f"Train set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Train period: {train_df['date'].min()} to {train_df['date'].max()}")
print(f"Test period: {test_df['date'].min()} to {test_df['date'].max()}")

## 5. Train LightGBM Model

In [ ]:
# LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

# Create datasets
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data, categorical_feature=categorical_features)

# Train model
print("Training LightGBM model...")
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, test_data],
    valid_names=['train', 'test'],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=100)]
)

print("\nModel training completed")
print(f"Best iteration: {model.best_iteration}")
print(f"Best score: {model.best_score['test']['rmse']:.4f}")

## 6. Model Evaluation

In [ ]:
# Make predictions
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)

# Calculate metrics
def calculate_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
    
    print(f"\n{dataset_name} Metrics:")
    print(f"  MAE:  {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2:   {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}

train_metrics = calculate_metrics(y_train, y_pred_train, "Train")
test_metrics = calculate_metrics(y_test, y_pred_test, "Test")

## 7. Feature Importance

In [ ]:
# Plot feature importance
importance_df = pd.DataFrame({
    'feature': model.feature_name(),
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(20), x='importance', y='feature')
plt.title('Top 20 Feature Importance (LightGBM)', fontsize=14, fontweight='bold')
plt.xlabel('Importance (Gain)')
plt.tight_layout()
plt.show()

print("\nTop 10 Important Features:")
print(importance_df.head(10).to_string(index=False))

## 8. Visualize Predictions

In [ ]:
# Scatter plot: Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Train set
axes[0].scatter(y_train, y_pred_train, alpha=0.3, s=10)
axes[0].plot([0, 100], [0, 100], 'r--', lw=2)
axes[0].set_xlabel('Actual Interest Value')
axes[0].set_ylabel('Predicted Interest Value')
axes[0].set_title(f'Train Set (R2 = {train_metrics["R2"]:.4f})')
axes[0].grid(True, alpha=0.3)

# Test set
axes[1].scatter(y_test, y_pred_test, alpha=0.3, s=10, color='orange')
axes[1].plot([0, 100], [0, 100], 'r--', lw=2)
axes[1].set_xlabel('Actual Interest Value')
axes[1].set_ylabel('Predicted Interest Value')
axes[1].set_title(f'Test Set (R2 = {test_metrics["R2"]:.4f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Time series plot for a sample keyword
sample_keyword = test_df['keyword'].iloc[0]
sample_category = test_df['category'].iloc[0]

sample_data = test_df[
    (test_df['keyword'] == sample_keyword) & 
    (test_df['category'] == sample_category)
].copy()

sample_data['predicted'] = model.predict(
    sample_data[feature_cols + categorical_features],
    num_iteration=model.best_iteration
)

plt.figure(figsize=(14, 6))
plt.plot(sample_data['date'], sample_data['interest_value'], label='Actual', marker='o', linewidth=2)
plt.plot(sample_data['date'], sample_data['predicted'], label='Predicted', marker='s', linewidth=2, alpha=0.7)
plt.title(f'Predictions for "{sample_keyword}" ({sample_category})', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Interest Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Save Model

In [ ]:
# Save model
model_path = '../models/lightgbm_daily_model.txt'
model.save_model(model_path)

# Save metrics
metrics = {
    'train': train_metrics,
    'test': test_metrics,
    'feature_importance': importance_df.to_dict('records')
}

import json
with open('../models/lightgbm_daily_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Model saved to: {model_path}")
print(f"Metrics saved to: ../models/lightgbm_daily_metrics.json")

## 10. Summary

### Model Performance:
- Algorithm: LightGBM
- Test RMSE: Check output above
- Test R2: Check output above
- Training Time: Fast (typically under 1 minute)

### Next Steps:
1. Try hyperparameter tuning with Optuna
2. Add more features (holidays, events, etc.)
3. Ensemble with other models
4. Deploy for production inference